# SETUP

In [ ]:
! apt-get install gcc g++

In [ ]:
%env PYTHONPATH=

! wget https://repo.anaconda.com/miniconda/Miniconda3-py38_23.3.1-0-Linux-x86_64.sh
! chmod +x Miniconda3-py38_23.3.1-0-Linux-x86_64.sh
! bash ./Miniconda3-py38_23.3.1-0-Linux-x86_64.sh -b -f -p /usr/local/
! rm -rf sample_data Miniconda3-py38_23.3.1-0-Linux-x86_64.sh

In [ ]:
! pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
! conda install pytorch pytorch-cuda=11.8 cudatoolkit=11.8 pytorch -c pytorch -c nvidia

In [ ]:
! pip install -U openmim
! mim install mmengine
! mim install "mmcv>=2.0.0"
! mim install "mmdet>=3.0.0"

In [ ]:
! git clone https://github.com/dany-el92/Action_Recognition.git
%cd Action_Recognition

# Preparazione Dataset

Aggiornare la cartella ctr-gcn-extended/data/ntu120/runs

In [ ]:
%cd ctr-gcn-extended/data/ntu120
! wget https://download.openmmlab.com/mmaction/posec3d/ntu120_xsub_train.pkl https://download.openmmlab.com/mmaction/posec3d/ntu120_xsub_val.pkl
! python gen_statistics.py
! python get_raw_skes_data.py
! python get_raw_denoised_data.py
! python seq_transformation.py

In [ ]:
! python main.py --config config/nturgbd120-cross-subject/default.yaml --work-dir work_dir/ntu120/csub/ctrgcn --device 0

# Training

Aggiornare ctr-gcn-extended/config/nturgbd120-cross-subject/default.yaml o altro file config (se utilizzato un nuovo file, modificare il parametro config)

In [10]:
%cd ../../

/content/progetto/degano-progetto-sysag-main/ctr-gcn-extended


In [ ]:
! pip install -r requirements.txt
! pip install -e torchlight
! pip install numba

In [ ]:
! python main.py --config config/nturgbd120-cross-subject/default.yaml --work-dir work_dir/ntu120/csub/ctrgcn --device 0

# Test

Aggiornare ctr-gcn-extended/config/nturgbd120-cross-subject/default.yaml o altro file config (se utilizzato un nuovo file, modificare il parametro config).\
Modificare il parametro weights

In [ ]:
! python main.py --config work_dir/ntu120/csub/ctrgcn/config.yaml --work-dir work_dir/ntu120/csub/ctrgcn --phase test --save-score True --weights work_dir/ntu120/csub/ctrgcn/runs-100-16300.pt --device 0

Aggiornare il path del file csv

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

num_classes = 120

confusion = np.loadtxt("work_dir/ntu120/csub/ctrgcn/epoch1_test_each_class_acc.csv",
                 delimiter=",", dtype=float)
accuracy = confusion[0]
confusion = confusion[1:]

print("Accuracy", sum(accuracy)/120)

confusion_df = pd.DataFrame(confusion, index=range(1, num_classes + 1), columns=range(1, num_classes + 1))
axes = sns.heatmap(confusion_df, cmap='nipy_spectral_r')
axes.figure.savefig('output_figure.png',dpi=600)